In [2]:
import pandas as pd
import numpy as np

# Simulated dataset
np.random.seed(0)
num_samples = 1000

df = pd.DataFrame({
    "route_id": np.arange(num_samples),
    "distance_km": np.random.uniform(10, 500, num_samples),
    "fuel_liters": np.random.uniform(5, 100, num_samples),
    "temperature": np.random.uniform(-10, 40, num_samples),
    "humidity": np.random.uniform(20, 100, num_samples),
    "weather_condition": np.random.choice(["clear", "rain", "snow", "fog"], num_samples),
    "traffic_level": np.random.choice(["low", "medium", "high"], num_samples),
    "cargo_weight": np.random.uniform(100, 20000, num_samples),
})

# Simulated target variable (carbon emission in kg)
df["carbon_emission_kg"] = (
    0.02 * df["distance_km"] +
    0.5 * df["fuel_liters"] +
    0.01 * df["cargo_weight"] +
    np.random.normal(0, 5, num_samples)
)
df.head()


,route_id,distance_km,fuel_liters,temperature,humidity,weather_condition,traffic_level,cargo_weight,carbon_emission_kg
0,0,278.918617,61.323626,30.575924,53.117000,clear,medium,6411.657718,98.584494
1,1,360.442790,5.956051,13.804199,70.369464,rain,high,10633.133355,110.789686
2,2,305.354054,50.203489,16.157799,82.286741,rain,low,18680.294086,220.621863
3,3,276.992760,72.333187,2.526029,88.124624,rain,medium,11376.922485,155.041871
4,4,217.590852,9.177666,20.252151,85.313017,rain,low,5179.065788,65.741151


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Encode categorical variables
le_weather = LabelEncoder()
le_traffic = LabelEncoder()
df["weather_condition"] = le_weather.fit_transform(df["weather_condition"])
df["traffic_level"] = le_traffic.fit_transform(df["traffic_level"])

# Define features and target
features = ["distance_km", "fuel_liters", "temperature", "humidity",
            "weather_condition", "traffic_level", "cargo_weight"]
target = "carbon_emission_kg"

# Normalize features
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)


Epoch 1/50


C:\Users\palguni\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 21646.2637 - mae: 135.0279 - val_loss: 21340.3262 - val_mae: 133.3448
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 21507.7324 - mae: 134.0200 - val_loss: 20905.2090 - val_mae: 131.7808
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 20108.9199 - mae: 129.8931 - val_loss: 20218.5742 - val_mae: 129.3148
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19720.4668 - mae: 127.9659 - val_loss: 19173.9121 - val_mae: 125.5292
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 18044.1523 - mae: 121.8567 - val_loss: 17693.4062 - val_mae: 120.0388
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 17400.5879 - mae: 118.6374 - val_loss: 15694.1484 - val_mae: 112.4182
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 14901.1143 - mae: 109.7443 - val_loss: 13214.0625 - val_mae: 102.3278
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 11626.3740 - mae: 95.3290 - val_loss: 10426.7715 - val_m

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test).flatten()

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
percent_error = (mae / y_test.mean()) * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"% Error: {percent_error:.2f}%")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
MAE: 10.37
RMSE: 12.98
% Error: 7.24%


C:\Users\palguni\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
def suggest_greenest_route(possible_routes_df, live_conditions):
    # Prepare input features
    live_data = possible_routes_df.copy()
    for key, value in live_conditions.items():
        live_data[key] = value

    # Encode and scale
    live_data["weather_condition"] = le_weather.transform(live_data["weather_condition"])
    live_data["traffic_level"] = le_traffic.transform(live_data["traffic_level"])
    live_data[features] = scaler.transform(live_data[features])

    # Predict emissions
    live_data["predicted_emission"] = model.predict(live_data[features]).flatten()

    # Suggest route with minimum emission
    return live_data.sort_values(by="predicted_emission").head(1)

# Example use
new_routes = pd.DataFrame({
    "distance_km": [50, 70],
    "fuel_liters": [10, 12],
    "temperature": [25, 25],
    "humidity": [60, 60],
    "weather_condition": ["clear", "clear"],
    "traffic_level": ["medium", "high"],
    "cargo_weight": [500, 500]
})
suggest_greenest_route(new_routes, {})


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


,distance_km,fuel_liters,temperature,humidity,weather_condition,traffic_level,cargo_weight,predicted_emission
0,-1.425638,-1.53439,0.678483,0.056988,-1.370105,1.176438,-1.728185,49.577557
